In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install fsspec==2023.9.2
    !pip install -U transformers trl
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth


In [2]:
from datasets import load_dataset
#dataset = load_dataset("deebak14/rhinoscript_ft_data_04", split = "train")

train_dataset = load_dataset("deebak14/rhinoscript_ft_data_04", split = "train")
eval_dataset = load_dataset("deebak14/rhinoscript_ft_data_04_test", split = "train")


README.md:   0%|          | 0.00/346 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/173k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1154 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/342 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/84.6k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/463 [00:00<?, ? examples/s]

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

model_name = "Qwen/Qwen3-8B"            #"Qwen/Qwen2.5-Coder-7B-Instruct" #"mistralai/Mistral-7B-Instruct-v0.3" #"google/gemma-3-12b-it"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Setup bitsandbytes 4-bit config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

# Load the model in 4bit
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",         # auto-assigns model to GPU if available
    torch_dtype=torch.float16, # or 'auto' if you want auto-detection
)

# Set max sequence length when tokenizing/generating, not in .from_pretrained
max_seq_length = 4096


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

...

model-00004-of-00005.safetensors:   0%|          | 0.00/3.19G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [5]:
# Before adding new tokens
print("Tokenizer vocab size before:", len(tokenizer))

# Add new tokens
new_tokens = [
    "CurvePlane",
    "CurvePointCount",
    "CurvePoints",
    "CurveRadius",
    "CurveSeam",
    "CurveStartPoint",
    "CurveSurfaceIntersection",
    "CurveTangent",
    "CurveWeights",
    "DivideCurve",
    "DivideCurveEquidistant",
    "DivideCurveLength",
    "EllipseCenterPoint",
    "EllipseQuadPoints",
    "EvaluateCurve",
    "ExplodeCurves",
    "ExtendCurve",
    "ExtendCurveLength",
    "ExtendCurvePoint",
    "FairCurve",
    "FitCurve",
    "InsertCurveKnot",
    "IsArc",
    "IsCircle",
    "IsCurve",
    "IsCurveClosable",
    "IsCurveClosed",
    "IsCurveInPlane",
    "CurveArea",
    "CurveAreaCentroid",
    "CurveArrows",
    "CurveBooleanDifference",
    "CurveBooleanIntersection",
    "CurveBooleanUnion",
    "CurveBrepIntersect",
    "CurveClosestObject",
    "CurveClosestPoint",
    "CurveContourPoints",
    "CurveCurvature",
    "CurveCurveIntersection",
    "CurveDegree",
    "CurveDeviation",
    "CurveDim",
    "CurveDirectionsMatch",
    "CurveDiscontinuity",
    "CurveDomain",
    "CurveEditPoints",
    "CurveEndPoint",
    "CurveFilletPoints",
    "CurveFrame",
    "CurveKnotCount",
    "CurveKnots",
    "CurveLength",
    "CurveMidPoint",
    "CurveNormal",
    "CurveNormalizedParameter",
    "CurveParameter",
    "CurvePerpFrame",
    "IsCurveLinear",
    "IsCurvePeriodic",
    "IsCurvePlanar",
    "IsCurveRational",
    "IsEllipse",
    "IsLine",
    "IsPointOnCurve",
    "IsPolyCurve",
    "IsPolyline",
    "JoinCurves",
    "LineFitFromPoints",
    "MakeCurveNonPeriodic",
    "MeanCurve",
    "MeshPolyline",
    "OffsetCurve",
    "OffsetCurveOnSurface",
    "PlanarClosedCurveContainment",
    "PlanarCurveCollision",
    "PointInPlanarClosedCurve",
    "PolyCurveCount",
    "PolylineVertices",
    "ProjectCurveToMesh",
    "ProjectCurveToSurface",
    "RebuildCurve",
    "RemoveCurveKnot",
    "ReverseCurve",
    "SimplifyCurve",
    "SplitCurve",
    "TrimCurve",
    "ChangeCurveDegree",
    "AddTweenCurves",
    "AddArc",
    "AddArc3Pt",
    "AddArcPtTanPt",
    "AddBlendCurve",
    "AddCircle",
    "AddCircle3Pt",
    "AddCurve",
    "AddEllipse",
    "AddEllipse3Pt",
    "AddFilletCurve",
    "AddInterpCrvOnSrf",
    "AddInterpCrvOnSrfUV",
    "AddInterpCurve",
    "AddLine",
    "AddNurbsCurve",
    "AddPolyline",
    "AddRectangle",
    "AddSpiral",
    "AddSubCrv",
    "ArcAngle",
    "ArcCenterPoint",
    "ArcMidPoint",
    "ArcRadius",
    "CircleCenterPoint",
    "CircleCircumference",
    "CircleRadius",
    "CloseCurve",
    "ClosedCurveOrientation",
    "ConvertCurveToPolyline",
    "CurveArcLengthPoint",
    "AddClippingPlane",
    "AddPictureFrame",
    "AddPoint",
    "AddPointCloud",
    "AddPoints",
    "AddText",
    "AddTextDot",
    "Area",
    "BoundingBox",
    "CompareGeometry",
    "ExplodeText",
    "IsClippingPlane",
    "IsPoint",
    "IsPointCloud",
    "IsText",
    "IsTextDot",
    "PointCloudCount",
    "PointCloudHasHiddenPoints",
    "PointCloudHasPointColors",
    "PointCloudHidePoints",
    "PointCloudPointColors",
    "PointCloudPoints",
    "PointCloudKNeighbors",
    "PointCloudClosestPoints",
    "PointCoordinates",
    "TextDotFont",
    "TextDotHeight",
    "TextDotPoint",
    "TextDotText",
    "TextObjectFont",
    "TextObjectHeight",
    "TextObjectPlane",
    "TextObjectPoint",
    "TextObjectStyle",
    "TextObjectText",
    "LineClosestPoint",
    "LineCylinderIntersection",
    "LineIsFartherThan",
    "LineLineIntersection",
    "LineMaxDistanceTo",
    "LineMinDistanceTo",
    "LinePlane",
    "LinePlaneIntersection",
    "LineSphereIntersection",
    "LineTransform",
    "IsLinetype",
    "IsLinetypeReference",
    "LinetypeCount",
    "LinetypeNames",
    "DistanceToPlane",
    "EvaluatePlane",
    "IntersectPlanes",
    "MovePlane",
    "PlaneClosestPoint",
    "PlaneCurveIntersection",
    "PlaneEquation",
    "PlaneFitFromPoints",
    "PlaneFromFrame",
    "PlaneFromNormal",
    "PlaneFromPoints",
    "PlanePlaneIntersection",
    "PlaneSphereIntersection",
    "PlaneTransform",
    "RotatePlane",
    "WorldXYPlane",
    "WorldYZPlane",
    "WorldZXPlane",
    "IsVectorParallelTo",
    "IsVectorPerpendicularTo",
    "IsVectorTiny",
    "IsVectorZero",
    "PointAdd",
    "PointArrayClosestPoint",
    "PointArrayTransform",
    "PointClosestObject",
    "PointCompare",
    "PointDivide",
    "PointsAreCoplanar",
    "PointScale",
    "PointSubtract",
    "PointTransform",
    "ProjectPointToMesh",
    "ProjectPointToSurface",
    "PullPoints",
    "VectorAdd",
    "VectorAngle",
    "VectorCompare",
    "VectorCreate",
    "VectorCrossProduct",
    "VectorDivide",
    "VectorDotProduct",
    "VectorLength",
    "VectorMultiply",
    "VectorReverse",
    "VectorRotate",
    "VectorScale",
    "VectorSubtract",
    "VectorTransform",
    "VectorUnitize",
    "PointArrayBoundingBox",
    "IsSphere",
    "IsSurface",
    "IsSurfaceClosed",
    "IsSurfacePeriodic",
    "IsSurfacePlanar",
    "IsSurfaceRational",
    "IsSurfaceSingular",
    "IsSurfaceTrimmed",
    "IsTorus",
    "SurfaceSphere",
    "JoinSurfaces",
    "MakeSurfacePeriodic",
    "OffsetSurface",
    "PullCurve",
    "RebuildSurface",
    "RemoveSurfaceKnot",
    "ReverseSurface",
    "ShootRay",
    "ShortPath",
    "ShrinkTrimmedSurface",
    "SplitBrep",
    "SurfaceArea",
    "SurfaceAreaCentroid",
    "SurfaceAreaMoments",
    "SurfaceClosestPoint",
    "BrepClosestPoint",
    "CapPlanarHoles",
    "DuplicateEdgeCurves",
    "DuplicateSurfaceBorder",
    "EvaluateSurface",
    "ExtendSurface",
    "ExplodePolysurfaces",
    "ExtractIsoCurve",
    "ExtractSurface",
    "ExtrudeCurve",
    "ExtrudeCurvePoint",
    "ExtrudeCurveStraight",
    "ExtrudeSurface",
    "FilletSurfaces",
    "FlipSurface",
    "IntersectBreps",
    "IntersectSpheres",
    "IsBrep",
    "IsCone",
    "IsCylinder",
    "IsPlaneSurface",
    "IsPointInSurface",
    "IsPointOnSurface",
    "IsPolysurface",
    "IsPolysurfaceClosed",
    "SurfaceCone",
    "SurfaceCurvature",
    "SurfaceCylinder",
    "SurfaceDegree",
    "SurfaceDomain",
    "SurfaceEditPoints",
    "SurfaceEvaluate",
    "SurfaceFrame",
    "SurfaceIsocurveDensity",
    "SurfaceKnotCount",
    "SurfaceKnots",
    "SurfaceNormal",
    "SurfaceNormalizedParameter",
    "SurfaceParameter",
    "SurfacePointCount",
    "SurfacePoints",
    "SurfaceTorus",
    "SurfaceVolume",
    "SurfaceVolumeCentroid",
    "SurfaceVolumeMoments",
    "SurfaceWeights",
    "TrimBrep",
    "TrimSurface",
    "UnrollSurface",
    "ChangeSurfaceDegree",
    "AddBox",
    "AddCone",
    "AddCutPlane",
    "AddCylinder",
    "AddEdgeSrf",
    "AddNetworkSrf",
    "AddNurbsSurface",
    "AddPatch",
    "AddPipe",
    "AddPlanarSrf",
    "AddPlaneSurface",
    "AddLoftSrf",
    "AddRevSrf",
    "AddSphere",
    "AddSrfContourCrvs",
    "AddSrfControlPtGrid",
    "AddSrfPt",
    "AddSrfPtGrid",
    "AddSweep1",
    "AddSweep2",
    "AddRailRevSrf",
    "AddTorus",
    "BooleanDifference",
    "BooleanIntersection",
    "BooleanUnion",
    "IsXformIdentity",
    "IsXformSimilarity",
    "IsXformZero",
    "XformChangeBasis",
    "XformChangeBasis2",
    "XformCompare",
    "XformCPlaneToWorld",
    "XformDeterminant",
    "XformDiagonal",
    "XformIdentity",
    "XformInverse",
    "XformMirror",
    "XformMultiply",
    "XformPlanarProjection",
    "XformRotation1",
    "XformRotation2",
    "XformRotation3",
    "XformRotation4",
    "XformScale",
    "XformScreenToWorld",
    "XformShear",
    "XformTranslation",
    "XformWorldToCPlane",
    "XformWorldToScreen",
    "XformZero",
    "ContextIsRhino",
    "ContextIsGrasshopper",
    "Angle",
    "Angle2",
    "ClipboardText",
    "ColorAdjustLuma",
    "ColorBlueValue",
    "ColorGreenValue",
    "ColorHLSToRGB",
    "ColorRedValue",
    "ColorRGBToHLS",
    "CullDuplicateNumbers",
    "CullDuplicatePoints",
    "Distance",
    "GetSettings",
    "Polar",
    "SimplifyArray",
    "Sleep",
    "SortPointList",
    "SortPoints",
    "Str2Pt",
    "CreatePoint",
    "CreateVector",
    "CreatePlane",
    "CreateXform",
    "CreateColor",
    "CreateInterval",
    "MoveObject",
    "MoveObjects",
    "ObjectColor",
    "ObjectColorSource",
    "ObjectDescription",
    "ObjectGroups",
    "ObjectLayer",
    "ObjectLayout",
    "ObjectLinetype",
    "ObjectLinetypeSource",
    "ObjectMaterialIndex",
    "ObjectMaterialSource",
    "ObjectName",
    "ObjectPrintColor",
    "ObjectPrintColorSource",
    "ObjectPrintWidth",
    "ObjectPrintWidthSource",
    "ObjectType",
    "OrientObject",
    "RotateObject",
    "RotateObjects",
    "ScaleObject",
    "ScaleObjects",
    "SelectObject",
    "SelectObjects",
    "ShearObject",
    "ShearObjects",
    "ShowObject",
    "ShowObjects",
    "TransformObject",
    "TransformObjects",
    "UnlockObject",
    "UnlockObjects",
    "UnselectObject",
    "UnselectObjects",
    "CopyObject",
    "CopyObjects",
    "DeleteObject",
    "DeleteObjects",
    "FlashObject",
    "HideObject",
    "HideObjects",
    "IsLayoutObject",
    "IsObject",
    "IsObjectHidden",
    "IsObjectInBox",
    "IsObjectInGroup",
    "IsObjectLocked",
    "IsObjectNormal",
    "IsObjectReference",
    "IsObjectSelectable",
    "IsObjectSelected",
    "IsObjectSolid",
    "IsObjectValid",
    "IsVisibleInView",
    "LockObject",
    "LockObjects",
    "MatchObjectAttributes",
    "MirrorObject",
    "MirrorObjects",
    "AddMesh",
    "AddPlanarMesh",
    "CurveMeshIntersection",
    "DisjointMeshCount",
    "DuplicateMeshBorder",
    "ExplodeMeshes",
    "IsMesh",
    "IsMeshClosed",
    "IsMeshManifold",
    "IsPointOnMesh",
    "JoinMeshes",
    "MeshArea",
    "MeshAreaCentroid",
    "MeshBooleanDifference",
    "MeshBooleanIntersection",
    "MeshBooleanSplit",
    "MeshBooleanUnion",
    "MeshClosestPoint",
    "MeshFaceCenters",
    "MeshFaceCount",
    "MeshFaceNormals",
    "MeshFaces",
    "MeshFaceVertices",
    "MeshHasFaceNormals",
    "MeshHasTextureCoordinates",
    "MeshHasVertexColors",
    "MeshHasVertexNormals",
    "MeshMeshIntersection",
    "MeshNakedEdgePoints",
    "MeshOffset",
    "MeshOutline",
    "MeshQuadCount",
    "MeshQuadsToTriangles",
    "MeshToNurb",
    "MeshTriangleCount",
    "MeshVertexColors",
    "MeshVertexCount",
    "MeshVertexFaces",
    "MeshVertexNormals",
    "MeshVertices",
    "MeshVolume",
    "MeshVolumeCentroid",
    "PullCurveToMesh",
    "SplitDisjointMesh",
    "UnifyMeshNormals"
]
num_new_tokens = tokenizer.add_tokens(new_tokens)
print(f"Number of new tokens added: {num_new_tokens}")

# After adding new tokens
print("Tokenizer vocab size after:", len(tokenizer))
model.resize_token_embeddings(len(tokenizer))


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Tokenizer vocab size before: 128256
Number of new tokens added: 476
Tokenizer vocab size after: 128725


Embedding(128725, 2048)

In [4]:
from peft import LoraConfig, get_peft_model

# Configure LoRA
lora_config = LoraConfig(
    r=4,                     # LoRA rank
    lora_alpha=8,           # LoRA alpha (often 2x r)
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.01,
    bias="none",
    task_type="CAUSAL_LM",    # Important for language modeling
)

# Attach LoRA to model
model = get_peft_model(model, lora_config)

# (Optional) Print trainable parameters to confirm LoRA is applied
model.print_trainable_parameters()

trainable params: 10,911,744 || all params: 8,201,647,104 || trainable%: 0.1330


In [6]:
with open("/content/qwen_chat_template.jinja") as f:
    tokenizer.chat_template = f.read()

In [22]:
from trl import apply_chat_template

# Pick a few examples from your dataset
for i in range(3):
    sample = train_dataset[i]
    # Apply the chat template with tokenization and assistant mask
    output = tokenizer.apply_chat_template(
        sample['messages'],
        tokenize=True,
        return_assistant_tokens_mask=True,
        return_dict=True,
    )

    print(f"Sample {i}:")
    print("".join(map(str, output["assistant_masks"])))
    print("-" * 40)

Sample 0:
0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000011111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111
----------------------------------------
Sample 1:
0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001111111111111111111111111111111111111111111111
----------------------------------------
Sample 2:
000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111
----------------------------------------


In [7]:
from trl import SFTTrainer, SFTConfig

sft_config = SFTConfig(
    dataset_text_field="messages",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_ratio=0.05,                # Increased for more stable start
    num_train_epochs=10,
    learning_rate= 2e-6,             #1e-5             # Lowered for better generalization
    logging_steps=10,
    optim="adamw_8bit",
    weight_decay=0.05,
    lr_scheduler_type="cosine",
    seed=3407,
    report_to="none",
    eval_strategy="epoch",          # Evaluate and save every epoch
    save_strategy="epoch",
    save_total_limit=2,             # Only keep last 3 checkpoints
    max_grad_norm=1.0,              # Clip gradients
    fp16=True,                      # Enable mixed-precision if available (optional, can remove if not supported)
    push_to_hub=False,
    neftune_noise_alpha=5,
    assistant_only_loss=True,
    chat_template_path="/content/qwen_chat_template.jinja"
)

trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)


Tokenizing train dataset:   0%|          | 0/1154 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1154 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/463 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/463 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [8]:
# Get the full input_ids and assistant_masks for a sample
sample = trainer.train_dataset[100]
input_ids = sample["input_ids"]
assistant_masks = sample["assistant_masks"]

# Extract token ids for assistant tokens only
assistant_token_ids = [tid for tid, mask in zip(input_ids, assistant_masks) if mask == 1]

# Now decode ONLY those assistant tokens
assistant_text = tokenizer.decode(assistant_token_ids)
print("Assistant text only:\n", assistant_text)


Assistant text only:
 <|im_start|>assistant
<think>

</think>

rs.CurveBooleanSubtract is not a valid function. To calculate the difference between two closed, planar curves, use rs.CurveBooleanDifference(curve_id_0, curve_id_1, tolerance).



In [31]:
tokenizer.decode(trainer.train_dataset[100]["assistant_masks"])

'<unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s>'

In [9]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.557 GB.
7.773 GB of memory reserved.


Let's train the model! To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [10]:
trainer_stats = trainer.train()

Epoch,Training Loss,Validation Loss
1,2.577500,2.009559
2,1.788100,1.374618
3,1.215200,0.985788
4,1.050700,0.905273
5,0.923300,0.898811
6,0.895600,0.892996
7,0.928600,0.888767
8,0.921700,0.885900
9,0.895200,0.884881
10,0.868200,0.884631


In [8]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

2549.5129 seconds used for training.
42.49 minutes used for training.
Peak reserved memory = 9.822 GB.
Peak reserved memory for training = 2.927 GB.
Peak reserved memory % of max memory = 24.83 %.
Peak reserved memory for training % of max memory = 7.399 %.


<a name="Inference"></a>
### Inference
Let's run the model via Unsloth native inference! According to the `Qwen-3` team, the recommended settings for reasoning inference are `temperature = 0.6, top_p = 0.95, top_k = 20`

For normal chat based inference, `temperature = 0.7, top_p = 0.8, top_k = 20`

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-2.5",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
                {
                    "content": '''You are an expert in Python and Rhino 3D modeling using the rhinoscriptsyntax module and, if required, other native Python modules to accomplish the task. Your task is to interpret, analyze, and understand the user’s query, then respond with a corresponding Python script. Whenever you use a method from rhinoscriptsyntax, always ensure that it is an official part of the module, and that you use the correct number and types of arguments for each function. Your final code output should precisely capture the user’s intent, be free of ambiguity, and run without errors. You are an expert in Python and Rhino 3d modeling using rhinoscriptsyntax. Whenever you are asked to generate Python code based on a user query, your role is to carefully interpret and analyze the request and rigorously verify that every function and method you use is a valid part of the official rhinoscriptsyntax module, and ensure that the chosen method has the proper arguments and argument types. The resulting script should be reliable and free of errors''',
                    "role": "system"
                },
                {
                    "content": 'create cubes along a circle',
                    "role": "user"
                }
            ]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 2048, use_cache = True,
                         temperature = 0.6, top_p = 0.95, top_k = 20)
tokenizer.batch_decode(outputs)

NameError: name 'FastLanguageModel' is not defined

# Before SFT

In [3]:
from transformers import pipeline

# Use a pipeline for easy inference
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Example usage
messages = [
    {"role" : "system",
     "content" : """You are an expert in Python and Rhino 3D modeling using the rhinoscriptsyntax module and, if required, other native Python modules to accomplish the task. Your task is to interpret, analyze, and understand the user’s query, then respond with a corresponding Python script. Whenever you use a method from rhinoscriptsyntax, always ensure that it is an official part of the module, and that you use the correct number and types of arguments for each function. Your final code output should precisely capture the user’s intent, be free of ambiguity, and run without errors.\n\nOutput should be a python code inside ```python ``` block. """},
    {"role" : "user",
     "content" : "how to create a box."}
]

prompt = pipe.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking = True
)

outputs = pipe(
        prompt,
        max_new_tokens=2048,
        do_sample=True,
        temperature=0.6,
        top_p=0.95,
        top_k=20,
        min_p=0
)
print(outputs[0]["generated_text"])

NameError: name 'model' is not defined

# After SFT

In [12]:
from transformers import pipeline

# Use a pipeline for easy inference
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Example usage
messages = [
    {"role" : "system",
     "content" : """You are an expert in Python and scripting using the rhinoscriptsyntax module. Your task is to understand user query and generate a script accordingly to model in rhino 3d.
     Please reason step by step. Breakdown the user query into parts, based on which write an algorithm, then write the code. Do not prompt user for input in the code unless specified.
     Your Output should be a python code inside ```python ``` block. """},
    {"role" : "user",
     "content" : "create boxes along circle."}
]

prompt = pipe.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking = True
)

outputs = pipe(
        prompt,
        max_new_tokens=2048,
        do_sample=True,
        temperature=0.6,
        top_p=0.95,
        top_k=20,
        min_p=0
)
print(outputs[0]["generated_text"])

Device set to use cuda:0


KeyboardInterrupt: 

In [14]:
!pip install mistral_inference


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 71.8 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=c88299d22b5fc0f0f64c022314665c663cbdb7ef52a4b10fa8efe3946a3332c6
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire


In [15]:
from mistral_inference.transformer import Transformer
from mistral_inference.generate import generate

from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
from mistral_common.protocol.instruct.messages import UserMessage
from mistral_common.protocol.instruct.request import ChatCompletionRequest


tokenizer = tokenizer
model = model

completion_request = ChatCompletionRequest(messages=[UserMessage(content="How to create a box? create a python script using rhinoscriptsyntax and output code inisde ```python ``` block.")])

tokens = tokenizer.encode_chat_completion(completion_request).tokens

out_tokens, _ = generate([tokens], model, max_tokens=2048, temperature=0.0, eos_id=tokenizer.instruct_tokenizer.tokenizer.eos_id)
result = tokenizer.instruct_tokenizer.tokenizer.decode(out_tokens[0])

print(result)


AttributeError: LlamaTokenizerFast has no attribute encode_chat_completion

In [17]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = tokenizer
model = model

# Use TRL's helper or build manually
from trl import apply_chat_template
messages = [
    {"role": "user", "content": "How to create a box using rhinoscriptsyntax in Python? output code inisde ```python ``` block."},
]
prompt = apply_chat_template({"messages": messages}, tokenizer)["text"]

input_ids = tokenizer(prompt, return_tensors="pt").input_ids
output = model.generate(input_ids, max_new_tokens=256, temperature=0.0)
print(tokenizer.decode(output[0]))


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:2497: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by cal

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
messages = [
    {"role" : "system",
     "content" : """You are an expert in Python and Rhino 3D modeling using the rhinoscriptsyntax module. import rhinoscriptsyntax as rs. Only use functions from this module ```python\n ``` """},
    {"role" : "user",
     "content" : "create cubes along circle "}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = False, # Disable thinking
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 4096, # Increase for longer outputs!
    temperature = .6, top_p = 0.95, top_k = 20, # For thinking
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

To create cubes along a circle, use rs.AddCircle to create the base circle and rs.AddCylinder to create the cylinder. To create cubes, you can use rs.AddBox with a list of eight points for each corner of the cube.
<|im_end|>


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        load_in_4bit = True,
    )

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False:
    model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: # Pushing to HF Hub
    model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False:
    model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: # Pushing to HF Hub
    model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False:
    model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: # Pushing to HF Hub
    model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [ ]:
# Save to 8bit Q8_0
if False:
    model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False:
    model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False:
    model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: # Pushing to HF Hub
    model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False:
    model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: # Pushing to HF Hub
    model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
